In [1]:
%load_ext autoreload
%autoreload
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>
    div.output_area{
        max-height:10000px;
        overflow:scroll;
    }
</style>"""))

import shapefile
import pygeohash as geohash
from timeUtils import clock, elapsed
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point
from random import uniform
from geoUtils import *
basedir="/Users/tgadf/Downloads/osm"

Notebook Last Run Initiated: 2018-09-17 14:03:34.886932


In [4]:
from modelio import loadJoblib
buildingLookup = loadJoblib("buildingLookup.p")
buildingMap    = loadJoblib("buildingMap.p")
buildingConv   = loadJoblib("buildingConv.p")

def parseOSM(dirval):
    from os.path import basename, dirname, join, splitext
    from glob import glob
    from collections import Counter
    state = basename(dirval).replace("-latest-free.shp", "")
    print(state)
    print(dirval)
    useUnique = False
    osmpath = dirval
    osms     = [splitext(basename(x))[0] for x in glob(join(osmpath, "*build*.shp"))]
    blgCntr = Counter()
    nameCntr = Counter()
    nameLookup = {}
    for osm in osms:
        print(osm)
        shapeval = osm
        
        files = glob(join(basedir, "{0}-{1}-{2}*".format(state, osm, 7)))
        if len(files) > 0:
            print("Already processed this one...")
            #continue

        try:
            sf = shapefile.Reader(join(osmpath, osm))
        except:
            raise ValueError("No shapefile!")
        fields      = sf.fields
        shapeData   = {}
        geoShapeMap = {}
        Nshapes   = len(sf.shapes())
        ngeos     = 0
        totalgeos = 0
        prec      = 7
        show      = False
        cntr      = Counter()

        start,cmt = clock("\n\nAnalyzing {0}\t{1}".format(shapeval, Nshapes))
        if show:
            print("Fields -> {0}".format(fields))

        irec = -1
        for shapeRec in sf.iterShapeRecords():
            irec += 1
            if irec % 25000 == 0 and irec > 0:
                if True:
                    print("Processed {0}/{1} records. Found {2} geos so far...".format(irec, Nshapes, ngeos))

            ## Record
            record = shapeRec.record
            if show:
                print("Stopping at 1st record: {0}".format(record))
                break
            if osm in ["gis_osm_buildings_a_free_1"]:
                if useUnique:
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = "building"
                fclass = record[2]
                name   = record[3]
                ftype  = record[4]
                if buildingLookup.get(name) is not None:
                    fclass = buildingLookup[name]
                    geoid  = fclass
                else:
                    if len(ftype) > 0:
                        fclass = ftype
                        geoid  = fclass
                    else:
                        found=None
                        for k,v in buildingMap.items():
                            if k in name.split():
                                fclass = v
                                found  = v
                                geoid  = fclass
                                break
                            if name.find("{0} ".format(k)) != -1 or name.find(" {0}".format(k)) != -1:
                                fclass = v
                                found  = v
                                geoid  = fclass
                                break
                                
                        if found is None:
                            nameCntr[name] += 1
                            if nameLookup.get(name) is None:
                                if len(ftype) > 0:
                                    nameLookup[name] = ftype
                            for val in name.split():
                                blgCntr[val] += 1
                if buildingConv.get(geoid):
                    geoid = buildingConv[geoid]
                if buildingConv.get(fclass):
                    fclass = buildingConv[fclass]
            elif osm in ["gis_osm_pofw_a_free_1", "gis_osm_pofw_free_1", "gis_osm_pois_a_free_1", "gis_osm_pois_free_1", "gis_osm_railways_free_1"] or osm in ["gis_osm_traffic_a_free_1", "gis_osm_traffic_free_1", "gis_osm_transport_a_free_1", "gis_osm_transport_free_1"]:
                if useUnique:
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                name   = record[3]
            else:
                print("Not recording {0}".format(osm))
                break
#            elif osm in ["gis_osm_places_a_free_1", "gis_osm_places_free_1"]:
#                if useUnique:
#                    geoid  = record[0]
#                else:
#                    code   = record[1]
#                    geoid  = code
#                fclass = record[2]
#                pop    = record[3]
#                name   = record[4]

            shapeData[geoid] = {"Name": fclass, "Record": irec}
            cntr[geoid] += 1

            record = shapeRec.record
            shape  = shapeRec.shape
            points = shape.points
            useLinear = False
            if len(points) == 1 or useLinear is True:
                useLinear = True
                geos = addLinearGeos(irec, Nshapes, shape, prec, debug=False)
            else:
                geos = getInitGeo(shape)
                geos = addShapeGeos(irec, Nshapes, shape, prec, geos, debug=False)
                if geos is None:
                    useLinear = True
                    geos = addLinearGeos(irec, Nshapes, shape, prec, debug=False) 

            if geoShapeMap.get(geoid) is None:
                geoShapeMap[geoid] = {}
            for geo in geos:
                geoShapeMap[geoid][geo] = True
            #geoShapeMap[geoid] = geoShapeMap[geoid].union(geos)
            ngeos += len(geos)

        #print(cntr.most_common(100))
        found = False
        for k,v in cntr.most_common():            
            if k.find(" ") != -1 or k.find(";") != -1 or k[0].isupper():
                print("buildingConv[\"{0}\"] = \"{1}\"".format(k, k))
                found = True
        print("\n")
        for k,v in nameCntr.most_common(200):
            print("commerce[\"{0}\"] = \"{1}\"".format(k, nameLookup.get(k)))
        if len(shapeData) > 0:
            for geoid in geoShapeMap.keys():
                geoShapeMap[geoid] = list(geoShapeMap[geoid].keys())
            print("Found {0} geos from {1}".format(ngeos, shapeval))
            saveGeoData(shapeData, geoShapeMap, Nshapes, ngeos, join(basedir, "{0}-{1}-{2}".format(state, osm, 7)))

In [ ]:
import zipfile
from glob import glob
from os import mkdir
from os.path import splitext, basename, dirname, join, exists
zipfiles = glob(join(basedir, "[o-z]*.zip"))
for zipname in zipfiles:
    statedir = dirname(zipname)
    name     = splitext(basename(zipname))[0]
    dirval   = join(statedir, name) 
    if exists(dirval):
        continue
    try:
        mkdir(dirval)
    except:
        pass
    zip_ref = zipfile.ZipFile(zipname, 'r')
    zip_ref.extractall(dirval)
    
    try:
        parseOSM(dirval)
    except:
        pass
    
    zip_ref.close()
    
    rmZipDir(dirval)

ohio
/Users/tgadf/Downloads/osm/ohio-latest-free.shp
gis_osm_buildings_a_free_1
Already processed this one...
Current Time is Mon Sep 17, 2018 15:16:50 for 

Analyzing gis_osm_buildings_a_free_1	327413
Processed 25000/327413 records. Found 38437 geos so far...
Processed 50000/327413 records. Found 77008 geos so far...
Processed 75000/327413 records. Found 115619 geos so far...
Processed 100000/327413 records. Found 152866 geos so far...
Processed 125000/327413 records. Found 185986 geos so far...
Processed 150000/327413 records. Found 218908 geos so far...
Processed 175000/327413 records. Found 252888 geos so far...
Processed 200000/327413 records. Found 287496 geos so far...
Processed 225000/327413 records. Found 320861 geos so far...
Processed 250000/327413 records. Found 353101 geos so far...
Processed 275000/327413 records. Found 386470 geos so far...
Processed 300000/327413 records. Found 418844 geos so far...
Processed 325000/327413 records. Found 452413 geos so far...


commerce

oklahoma
/Users/tgadf/Downloads/osm/oklahoma-latest-free.shp
gis_osm_buildings_a_free_1
Already processed this one...
Current Time is Mon Sep 17, 2018 15:22:18 for 

Analyzing gis_osm_buildings_a_free_1	74273
Processed 25000/74273 records. Found 35773 geos so far...
Processed 50000/74273 records. Found 70710 geos so far...


commerce[""] = "None"
commerce["Love's"] = "None"
commerce["Reasor's"] = "None"
commerce["Physical Plant"] = "None"
commerce["Murphy USA"] = "None"
commerce["Sinclair"] = "None"
commerce["OnCue"] = "None"
commerce["Chesapeake Energy"] = "None"
commerce["Aircraft Hanger"] = "None"
commerce["Jimmy's Egg"] = "None"
commerce["On Cue"] = "None"
commerce["Club House"] = "None"
commerce["Mazzio's"] = "None"
commerce["Fiesta Mart"] = "None"
commerce["Golden Chick"] = "None"
commerce["Homeland"] = "None"
commerce["Grady County Fairgrounds"] = "None"
commerce["Brookdale"] = "None"
commerce["Kellyville"] = "None"
commerce["Central Plant"] = "None"
commerce["Rib Crib"] = "None

Processed 100000/697841 records. Found 132529 geos so far...
Processed 125000/697841 records. Found 164108 geos so far...
Processed 150000/697841 records. Found 195789 geos so far...
Processed 175000/697841 records. Found 228151 geos so far...
Processed 200000/697841 records. Found 260240 geos so far...
Processed 225000/697841 records. Found 291895 geos so far...
Processed 250000/697841 records. Found 323533 geos so far...
Processed 275000/697841 records. Found 355426 geos so far...
Processed 300000/697841 records. Found 386119 geos so far...
Processed 325000/697841 records. Found 415973 geos so far...
Processed 350000/697841 records. Found 446349 geos so far...
Processed 375000/697841 records. Found 477671 geos so far...
Processed 400000/697841 records. Found 508211 geos so far...
Processed 425000/697841 records. Found 538762 geos so far...
Processed 450000/697841 records. Found 570429 geos so far...
Processed 475000/697841 records. Found 601396 geos so far...
Processed 500000/697841 

pennsylvania
/Users/tgadf/Downloads/osm/pennsylvania-latest-free.shp
gis_osm_buildings_a_free_1
Already processed this one...
Current Time is Mon Sep 17, 2018 15:35:43 for 

Analyzing gis_osm_buildings_a_free_1	704279
Processed 25000/704279 records. Found 38169 geos so far...
Processed 50000/704279 records. Found 76219 geos so far...
Processed 75000/704279 records. Found 111813 geos so far...
Processed 100000/704279 records. Found 146319 geos so far...
Processed 125000/704279 records. Found 178965 geos so far...
Processed 150000/704279 records. Found 212674 geos so far...
Processed 175000/704279 records. Found 245534 geos so far...
Processed 200000/704279 records. Found 276988 geos so far...
Processed 225000/704279 records. Found 309254 geos so far...
Processed 250000/704279 records. Found 341357 geos so far...
Processed 275000/704279 records. Found 372622 geos so far...
Processed 300000/704279 records. Found 403409 geos so far...
Processed 325000/704279 records. Found 433225 geos so f

puerto-rico
/Users/tgadf/Downloads/osm/puerto-rico-latest-free.shp
gis_osm_buildings_a_free_1
Already processed this one...
Current Time is Mon Sep 17, 2018 15:47:22 for 

Analyzing gis_osm_buildings_a_free_1	1500126
Processed 25000/1500126 records. Found 32598 geos so far...
Processed 50000/1500126 records. Found 64100 geos so far...
Processed 75000/1500126 records. Found 95021 geos so far...
Processed 100000/1500126 records. Found 126187 geos so far...
Processed 125000/1500126 records. Found 157497 geos so far...
Processed 150000/1500126 records. Found 188909 geos so far...
Processed 175000/1500126 records. Found 220184 geos so far...
Processed 200000/1500126 records. Found 251078 geos so far...
Processed 225000/1500126 records. Found 282624 geos so far...
Processed 250000/1500126 records. Found 312923 geos so far...
Processed 275000/1500126 records. Found 342844 geos so far...
Processed 300000/1500126 records. Found 372557 geos so far...
Processed 325000/1500126 records. Found 40252

...

virginia
/Users/tgadf/Downloads/osm/virginia-latest-free.shp
gis_osm_buildings_a_free_1
Already processed this one...
Current Time is Mon Sep 17, 2018 16:52:17 for 

Analyzing gis_osm_buildings_a_free_1	846783
Processed 25000/846783 records. Found 38645 geos so far...
Processed 50000/846783 records. Found 75971 geos so far...
Processed 75000/846783 records. Found 110679 geos so far...
Processed 100000/846783 records. Found 142724 geos so far...
Processed 125000/846783 records. Found 173687 geos so far...
Processed 150000/846783 records. Found 204736 geos so far...
Processed 175000/846783 records. Found 236232 geos so far...
Processed 200000/846783 records. Found 268320 geos so far...
Processed 225000/846783 records. Found 299552 geos so far...
Processed 250000/846783 records. Found 329603 geos so far...
Processed 275000/846783 records. Found 360210 geos so far...
Processed 300000/846783 records. Found 391859 geos so far...
Processed 325000/846783 records. Found 424071 geos so far...
Pr

washington
/Users/tgadf/Downloads/osm/washington-latest-free.shp
gis_osm_buildings_a_free_1
Already processed this one...
Current Time is Mon Sep 17, 2018 17:06:46 for 

Analyzing gis_osm_buildings_a_free_1	468136
Processed 25000/468136 records. Found 40211 geos so far...
Processed 50000/468136 records. Found 71533 geos so far...
Processed 75000/468136 records. Found 102763 geos so far...
Processed 100000/468136 records. Found 133430 geos so far...
Processed 125000/468136 records. Found 164380 geos so far...
Processed 150000/468136 records. Found 195901 geos so far...
Processed 175000/468136 records. Found 226920 geos so far...
Processed 200000/468136 records. Found 258971 geos so far...
Processed 225000/468136 records. Found 289601 geos so far...
Processed 250000/468136 records. Found 323886 geos so far...
Processed 275000/468136 records. Found 357512 geos so far...
Processed 300000/468136 records. Found 390327 geos so far...
Processed 325000/468136 records. Found 424138 geos so far..

Current Time is Mon Sep 17, 2018 17:15:22 for 

Analyzing gis_osm_buildings_a_free_1	70700
Processed 25000/70700 records. Found 33623 geos so far...
Processed 50000/70700 records. Found 66530 geos so far...


commerce[""] = "None"
commerce["GoMart"] = "None"
commerce["WesBanco"] = "None"
commerce["Tudor's Biscuit World"] = "None"
commerce["Yokum's Vacationland Cabin"] = "None"
commerce["point of view farm"] = "None"
commerce["Roselawn Mausoleum"] = "None"
commerce["Microtel"] = "None"
commerce["Mausoleum"] = "None"
commerce["One Stop"] = "None"
commerce["Mattress Warehouse"] = "None"
commerce["Maintenance Building"] = "None"
commerce["Dorm"] = "None"
commerce["Camden Villa"] = "None"
commerce["Giant Eagle"] = "None"
commerce["Books-A-Million"] = "None"
commerce["Science Hall"] = "None"
commerce["Administration Building"] = "None"
commerce["Go-Mart"] = "None"
commerce["Foodland"] = "None"
commerce["BFS"] = "None"
commerce["Mario's Fishbowl"] = "None"
commerce["Guard Office"] = "None"
co

wisconsin
/Users/tgadf/Downloads/osm/wisconsin-latest-free.shp
gis_osm_buildings_a_free_1
Already processed this one...
Current Time is Mon Sep 17, 2018 17:16:45 for 

Analyzing gis_osm_buildings_a_free_1	223434
Processed 25000/223434 records. Found 37536 geos so far...
Processed 50000/223434 records. Found 72995 geos so far...
Processed 75000/223434 records. Found 107520 geos so far...
Processed 100000/223434 records. Found 141422 geos so far...
Processed 125000/223434 records. Found 173990 geos so far...
Processed 150000/223434 records. Found 207034 geos so far...
Processed 175000/223434 records. Found 240523 geos so far...
Processed 200000/223434 records. Found 272341 geos so far...


commerce[""] = "None"
commerce["Shopko"] = "None"
commerce["Festival Foods"] = "None"
commerce["Cenex"] = "None"
commerce["Park Office"] = "None"
commerce["PDQ"] = "None"
commerce["Holiday"] = "None"
commerce["Clubhouse"] = "None"
commerce["Pavillion"] = "None"
commerce["Shelter"] = "None"
commerce["Sk

wyoming
/Users/tgadf/Downloads/osm/wyoming-latest-free.shp
gis_osm_buildings_a_free_1
Already processed this one...
Current Time is Mon Sep 17, 2018 17:20:41 for 

Analyzing gis_osm_buildings_a_free_1	54570
Processed 25000/54570 records. Found 32107 geos so far...
Processed 50000/54570 records. Found 64998 geos so far...


commerce[""] = "None"
commerce["Outhouse"] = "None"
commerce["Wyoming Work Warehouse"] = "None"
commerce["Sierra Trading Post"] = "None"
commerce["trailer"] = "None"
commerce["Jackson Lumber"] = "None"
commerce["Old Chicago"] = "None"
commerce["Bloedorn Lumber"] = "None"
commerce["TCRI"] = "None"
commerce["Loaf 'N Jug"] = "None"
commerce["Trading Post"] = "None"
commerce["Vault Toilet"] = "None"
commerce["Clubhouse"] = "None"
commerce["WYDOT"] = "None"
commerce["Sterling Communications & Electronics"] = "None"
commerce["Maverik"] = "None"
commerce["Amphitheatre"] = "None"
commerce["Lower Hamilton Store"] = "None"
commerce["Executive House"] = "None"
commerce["Yellows